In [ ]:
!pip install torch
!pip install json
!pip install transformers
!pip install transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 5.0 MB/s eta 0:00:00


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoConfig
from transformers import TextDataset, DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments
import json

def load_dataset(file_path, tokenizer):
    with open(file_path, "r") as f:
        data = json.load(f)
    texts = [item["keywords"] + [""] + [item["description"]] for item in data]
    return TextDataset(
        tokenizer=tokenizer,
        file_path=file_path,
        block_size=128  # Adjust the block size as needed
    )

def main():
    # model_name = "allenai/longformer-base-4096"  # You can use other Longformer variants as well
    model_name = "mistralai/Mistral-7B-v0.1"  # You can use other Longformer variants as well
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(model_name)
    config = AutoConfig.from_pretrained(model_name)

    # Replace with your fine-tuning dataset
    train_dataset = load_dataset("product_descriptions.json", tokenizer)

    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer,
        mlm=True  # Now we are doing masked language modeling
    )

    training_args = TrainingArguments(
        output_dir="./fine-tuned-product-model-llm",
        overwrite_output_dir=True,
        num_train_epochs=3,
        per_device_train_batch_size=2,
        save_steps=10_000,
        save_total_limit=2,
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        data_collator=data_collator,
        train_dataset=train_dataset,
    )

    # Fine-tuning the model
    trainer.train()

    # Save the fine-tuned model
    model.save_pretrained("./fine-tuned-product-model-llm")
    tokenizer.save_pretrained("./fine-tuned-product-model-llm")

if __name__ == "__main__":
    main()


tokenizer_config.json:   0%|          | 0.00/966 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/5.06G [00:00<?, ?B/s]

In [ ]:
from transformers import AutoModelForMaskedLM, AutoTokenizer

def generate_product_description(model, tokenizer, keywords):
    input_text = " ".join(keywords) + " "
    input_ids = tokenizer.encode(input_text, return_tensors="pt")

    # Generate product description
    output = model.generate(input_ids, max_length=100, num_beams=5, no_repeat_ngram_size=2, top_k=50, top_p=0.95, temperature=0.7)

    # Decode and return the generated text
    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
    return generated_text

def main():
    # Load the fine-tuned model and tokenizer
    model_path = "./fine-tuned-product-model-llm"  # Adjust the path to your fine-tuned Longformer model
    model = AutoModelForMaskedLM.from_pretrained(model_path)
    tokenizer = AutoTokenizer.from_pretrained(model_path)

    # Keywords for testing
    test_keywords = ["lipstick", "vibrant", "long-lasting"]

    # Generate product description
    generated_description = generate_product_description(model, tokenizer, test_keywords)

    # Print the generated description
    print("Generated Product Description:")
    print(f"[{generated_description}]")

if __name__ == "__main__":
    main()
